In [1]:
import pandas as pd 
import numpy as np
import os 
import json

# Read in Data 

### Clean data for all time

In [2]:
data_name = "data.csv" 
df = pd.read_csv(data_name)

### Put together countries info 
- Pull in all the separate excel sheets to put together a country lookup 

In [3]:
data_name_raw = "CDP Community GHG Emissions 2016-2020.xlsx"

In [4]:
cols = ['Organization', 'City', 'Country', 'CDP Region', "City Location"]
cols2 = ['Organization', 'City', 'Country', 'Region', "City Location"]

In [5]:
df_2020 = pd.read_excel(data_name_raw, sheet_name="Received in 2020")
cities_2020 = df_2020[cols].drop_duplicates(subset=['Organization'])

df_2019 = pd.read_excel(data_name_raw, sheet_name="Received in 2019")
cities_2019 = df_2019[cols].drop_duplicates(subset=['Organization'])

df_2018 = pd.read_excel(data_name_raw, sheet_name="Received in 2018")
cities_2018 = df_2018[cols].drop_duplicates(subset=['Organization'])

df_2017 = pd.read_excel(data_name_raw, sheet_name="Received in 2017")
cities_2017 = df_2017[cols2].drop_duplicates(subset=['Organization'])
cities_2017.columns = cols


cities_all = pd.concat([cities_2017, cities_2018, cities_2019, cities_2020])
cities_all = cities_all.drop_duplicates(subset=['Organization'])

# rename because the final data has the organisation as city 
cities_all.columns = ['City', 'CityName', 'Country', 'Region', "City Location"]

# Autralia appears in 2 different places 
cities_all = cities_all[~((cities_all['Region'] == 'Southeast Asia and Oceania') & (cities_all['Country'] == 'Australia'))]


/Users/user/Library/Python/3.8/lib/python/site-packages/openpyxl/worksheet/_reader.py:300: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


## Data Cleaning 

the years should be integers & we don't need data where the year is missing

In [6]:
df = df.replace('2015-07-15 - 2017-12-30', np.nan)
df = df.dropna(subset=['Reporting Period'])

clean and convert columns that should be floats or ints

In [7]:
df['Reporting Period'] = df['Reporting Period'].astype(int)
df['Year Reported'] = df['Year Reported'].astype(int)

In [8]:
df['GHG/Capita'] = df['GHG/Capita']

In [9]:
df['GHG/Capita'] = df['GHG/Capita'].replace(
    {
        " -   ": -99,
        ' 6,144.982 ': -99,
        '#DIV/0!': -99,
        ' 1,956.061 ': -99,
        ' 1,385.454 ': -99,
        ' 2,625.293 ': -99,
        ' 5,818.133 ': -99,
        
    }
).astype(float)
df = df.loc[df['GHG/Capita'] != -99]

#### Add the cities info to the data 

In [10]:
df = df.set_index('City').join(cities_all.set_index('City')).reset_index()

#### Make sure that al the columns are the right type and fill missing strings with "missing" and missing numbers with -99

In [11]:
df['City'] = df['City'].fillna("missing")
df['Reporting Period'] = df['Reporting Period'].astype(int)#.fillna(-99)
df['Total emissions (metric tonnes CO2e)'] = df['Total emissions (metric tonnes CO2e)'].astype(float)#.fillna(-99)
df['GHG/Capita'] = df['GHG/Capita'].astype(float)#.fillna(-99)
df[' Population '] = df[' Population '].astype(float)#.fillna(-99)
df['Year Reported'] = df['Year Reported'].astype(int)#.fillna(-99)
df['Methodology/Protocol'] = df['Methodology/Protocol'].fillna("missing")
df['CityName'] = df['CityName'].fillna("missing")
df['Country'] = df['Country'].fillna("missing")
df['Region'] = df['Region'].fillna("missing")
df['City Location'] = df['City Location'].fillna("location")

Remove the duplicate country names

In [12]:
# This is to replace the duplicate country names 
# note that there are also other issues 
duplicate_country_names_dict = {
    'United Kingdom of Great Britain and Northern Ireland': 'United Kingdom',
    'United States of America': 'USA' 
}

In [13]:
df['Country'] = df['Country'].replace(duplicate_country_names_dict)

# Exploration 

In [14]:
len(df['City'].unique().tolist())

332

In [15]:
df['Region'].unique()

array(['Europe', 'South and West Asia', 'North America', 'Africa',
       'Latin America', 'missing', 'South Asia and Oceania', 'East Asia',
       'Southeast Asia and Oceania'], dtype=object)

In [16]:
df.dropna(subset=["Total emissions (metric tonnes CO2e)"]).groupby('Region')['Total emissions (metric tonnes CO2e)'].mean().sort_values(ascending=False)

Region
East Asia                     2.964347e+07
Africa                        1.688149e+07
South Asia and Oceania        7.359037e+06
Latin America                 7.358500e+06
missing                       7.024983e+06
North America                 6.564378e+06
South and West Asia           6.395928e+06
Europe                        4.344867e+06
Southeast Asia and Oceania    1.943736e+06
Name: Total emissions (metric tonnes CO2e), dtype: float64

#### Number of years
Observation: There is very little data that spans multiple years. Maybe it would make the most sense to consider only data in a certain period? e.g. 
- 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018 : would give us at most 20 cities to compare 
- 2013, 2014, 2015, 2016, 2017 : would give us at most 49 cities to compare

In [17]:
df['Reporting Period'].value_counts()

2015    133
2014    104
2016    100
2013     68
2017     48
2012     30
2010     22
2018     22
2011     13
2019      9
2009      6
2005      4
2007      3
2008      3
2006      1
Name: Reporting Period, dtype: int64

#### How many years do cities appear in 
Observations after the exploration
- There are 144 cities with 2 or more records
- There are 67 cities with 3 or more records
- There are 22 cities with 4 or more records
- There are only 6 cities with 5 records 
- There are no cities with more than 5 records
- Due to the very limited continuous data, it is best to not limit ourselves to a specific year range, and not to try to match the data so that we have only cities which all have data for the selected years. 
- We will instead take the top cities (e.g. 3 or more records or 4 or more records) regardless of which years these records are for 

In [18]:
cities_years = df.groupby('City')['Reporting Period'].count().sort_values(ascending=False)
cities_years = pd.DataFrame(cities_years).reset_index()
cities_years.columns = ['City', 'num_years']

In [19]:
cities_years

City  num_years
0                             Municipality of Curitiba          5
1                                      City of Calgary          5
2    Government of Hong Kong Special Administrative...          5
3                                       City of Sydney          5
4                                       City of Austin          5
..                                                 ...        ...
327                                    City of Roanoke          1
328                                  City of Rochester          1
329                                  City of San Diego          1
330                          City of Santa Barbara, CA          1
331                            Yilan County Government          1

[332 rows x 2 columns]

#### Extract cities based on how many records there are available for them 

In [20]:
def get_cities_by_number_years_reported(num_years_reported):
    cities = cities_years[cities_years['num_years']>=num_years_reported]['City'].tolist()
    df_cities = df.loc[df['City'].apply(lambda x : x in cities)]
    df_cities = df_cities.sort_values(by=['Total emissions (metric tonnes CO2e)'], ascending=False)
    return df_cities

In [21]:
df_cities_with_5_records = get_cities_by_number_years_reported(5)
df_cities_with_5_records.to_csv("cities_with_5_records.csv")

In [22]:
df_cities_with_4_records = get_cities_by_number_years_reported(4)
df_cities_with_4_records.to_csv("cities_with_4_records.csv")

In [23]:
df_cities_with_3_records = get_cities_by_number_years_reported(3)
df_cities_with_3_records.to_csv("cities_with_3_records.csv")

In [24]:
df_cities_with_2_records = get_cities_by_number_years_reported(2)
df_cities_with_2_records.to_csv("cities_with_2_records.csv")

## Tree Graph with all countries 
Transform the data into format for tree graph in d3 

In [26]:
def get_cities_data(df):
    emissions_total = pd.DataFrame(df.groupby('City')['Total emissions (metric tonnes CO2e)'].mean())
    emissions_per_capita = pd.DataFrame(df.groupby('City')['GHG/Capita'].mean())
    population = pd.DataFrame(df.groupby('City')[' Population '].mean())
    
    df_cities_all = df[['City', 'CityName', 'Country', 'Region']].dropna().drop_duplicates().set_index('City')
    df_cities_all = df_cities_all.join(emissions_total).join(emissions_per_capita).join(population)
    
    # drop anything with nan's for now 
    df_cities_all = df_cities_all.dropna()
    # drop all the cities with a missing coutnry for now 
    df_cities_all = df_cities_all[df_cities_all['Country'] != 'missing']
    # drop duplicate cities 
    df_cities_all = df_cities_all.drop_duplicates(subset=['CityName'])
    # sort by either total emissions or per capita 
    df_cities_all = df_cities_all.sort_values(by=["Total emissions (metric tonnes CO2e)"], ascending=False)
    
    
    return df_cities_all

In [27]:
def get_children_for_region(region, df_with_cities):
    region_countries = df_with_cities[df_with_cities['Region'] == region]['Country'].unique()
    countries_children = []
    for country in region_countries:
        df_country = df_with_cities.loc[df_with_cities['Country'] == country]
        cities_children = []
        for city in df_country['CityName'].tolist():
            cities_children.append({
                "name": city, 
                "nameCityRegion": df_country.loc[df_country['CityName'] == city].index[0],
                "emissions_total": df_country.loc[df_country['CityName'] == city]['Total emissions (metric tonnes CO2e)'][0],
                "emissions_per_capita": df_country.loc[df_country['CityName'] == city]['GHG/Capita'][0],
                "population": df_country.loc[df_country['CityName'] == city][' Population '][0],
            })
        countries_children.append({"name": country, "children": cities_children})
    return countries_children

In [28]:
regions = ['North America', 
           'Latin America', 
           'East Asia', 'South and West Asia', 'South Asia and Oceania', 'Southeast Asia and Oceania', 
           'Africa',
           'Europe'
          ]

#### All the cities, even with just 1 record

In [29]:
df_cities_all = get_cities_data(df)

In [30]:
df_cities_all['Total emissions (metric tonnes CO2e)'].max()

63452904.6

In [31]:
region_children = []
for region in regions:
    countries_children = get_children_for_region(region, df_cities_all)
    region_children.append({"name": region, "children": countries_children})
data_for_tree = {"name": "", "children": region_children}

In [32]:
with open('data_for_tree_v0.json', 'w') as outfile:
    json.dump(data_for_tree, outfile)

#### Cities with at least 2 records

In [33]:
df_cities_2records = get_cities_data(df_cities_with_2_records)

In [34]:
region_children = []
for region in regions:
    countries_children = get_children_for_region(region, df_cities_2records)
    region_children.append({"name": region, "children": countries_children})
data_for_tree = {"name": "", "children": region_children}

In [35]:
with open('data_for_tree_v1.json', 'w') as outfile:
    json.dump(data_for_tree, outfile)